In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("features_30_sec.csv")
df=df.iloc[:,1:]
df.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [3]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["encoded_label"]=le.fit_transform(df["label"])

In [4]:
import json

# Convert np.int32 → int
classes = {str(k): int(v) for k, v in zip(le.classes_, le.transform(le.classes_))}

with open("label_classes3.json", "w") as file:
    json.dump(classes, file)


In [5]:
X=df.drop(columns=["label","encoded_label"])
Y=df["encoded_label"]
X.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
1,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,...,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282
2,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,...,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025
3,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
4,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160


In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,Y,train_size=0.80,random_state=13)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled=scaler.transform(x_test)

In [8]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

# Define model parameter grids
param_grid = {
    "Random Forest": {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None]
    },
    "SVM": {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    },
    "KNN": {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    },
    "MLP": {
        'hidden_layer_sizes': [(128, 64), (128, 64, 32)],
        'alpha': [0.0001, 0.001],
        'activation': ['relu', 'tanh']
    }
}

# Map model names to initializers
model_initializers = {
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),  # soft voting needs predict_proba
    "KNN": KNeighborsClassifier(),
    "MLP": MLPClassifier(max_iter=300)
}

best_models = {}

# Tune each model with GridSearch
for name, params in param_grid.items():
    print(f"\n🔍 Tuning {name}...")

    model = model_initializers[name]

    grid = GridSearchCV(model, params, cv=5, n_jobs=-1, scoring='accuracy')
    grid.fit(X_train_scaled, y_train)

    print(f"✅ Best Params: {grid.best_params_}")
    print(f"📈 Best CV Score: {grid.best_score_:.3f}")

    best_models[name] = grid.best_estimator_

# 🧠 Use top 3 models in VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('rf', best_models["Random Forest"]),
        ('svm', best_models["SVM"]),
        ('mlp', best_models["MLP"])
    ],
    voting='soft'  # uses probabilities
)

# Train and evaluate VotingClassifier
voting_clf.fit(X_train_scaled, y_train)
y_pred = voting_clf.predict(X_test_scaled)

print("\n🧠 Voting Classifier (Soft Voting)")
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📊 Classification Report:\n", classification_report(y_test, y_pred))



🔍 Tuning Random Forest...
✅ Best Params: {'max_depth': 10, 'n_estimators': 100}
📈 Best CV Score: 0.782

🔍 Tuning SVM...
✅ Best Params: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
📈 Best CV Score: 0.761

🔍 Tuning KNN...
✅ Best Params: {'n_neighbors': 5, 'weights': 'distance'}
📈 Best CV Score: 0.698

🔍 Tuning MLP...


c:\Users\bscs2213\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


✅ Best Params: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 64)}
📈 Best CV Score: 0.755

🧠 Voting Classifier (Soft Voting)
✅ Accuracy: 0.775
📊 Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.62      0.67        21
           1       0.92      0.92      0.92        12
           2       0.85      0.77      0.81        22
           3       0.59      0.68      0.63        19
           4       0.88      0.71      0.79        21
           5       0.88      0.96      0.92        24
           6       0.88      0.95      0.91        22
           7       0.94      0.74      0.83        23
           8       0.70      0.74      0.72        19
           9       0.48      0.65      0.55        17

    accuracy                           0.78       200
   macro avg       0.78      0.77      0.77       200
weighted avg       0.79      0.78      0.78       200



c:\Users\bscs2213\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
